Task-1 (mPAP)

In [ ]:
import torch
import random
import numpy as np
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
import torch.nn as nn
import torch.optim as optim

# **Set device configuration**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# **Set seed for reproducibility**
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(123)

# Define file paths for the 12-lead ECG data and labels
lead_file_paths = {
    "LEAD_I": "../../Data_processing/data_aspire_PAP_1/LEAD_I.pt",
    "LEAD_II": "../../Data_processing/data_aspire_PAP_1/LEAD_II.pt",
    "LEAD_III": "../../Data_processing/data_aspire_PAP_1/LEAD_III.pt",
    "LEAD_aVR": "../../Data_processing/data_aspire_PAP_1/LEAD_aVR.pt",
    "LEAD_aVL": "../../Data_processing/data_aspire_PAP_1/LEAD_aVL.pt",
    "LEAD_aVF": "../../Data_processing/data_aspire_PAP_1/LEAD_aVF.pt",
    "LEAD_V1": "../../Data_processing/data_aspire_PAP_1/LEAD_V1.pt",
    "LEAD_V2": "../../Data_processing/data_aspire_PAP_1/LEAD_V2.pt",
    "LEAD_V3": "../../Data_processing/data_aspire_PAP_1/LEAD_V3.pt",
    "LEAD_V4": "../../Data_processing/data_aspire_PAP_1/LEAD_V4.pt",
    "LEAD_V5": "../../Data_processing/data_aspire_PAP_1/LEAD_V5.pt",
    "LEAD_V6": "../../Data_processing/data_aspire_PAP_1/LEAD_V6.pt"
}
labels_file_path = "../../Data_processing/data_aspire_PAP_1/labels.pt"


# Load all lead tensors and labels
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Ensure all leads have the same number of samples
sample_count = len(next(iter(ecg_lead_tensors.values())))
assert len(labels) == sample_count, "Mismatch between number of labels and samples."
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count, "All leads must have the same number of samples."

# **Dataset Class**
class ECGMultiLeadDatasetWithLabels(Dataset):
    def __init__(self, ecg_leads, labels):
        self.ecg_leads = ecg_leads
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        lead_data = {lead: self.ecg_leads[lead][idx].unsqueeze(0) for lead in self.ecg_leads}
        label = self.labels[idx]
        return lead_data, label

# **Initialize the dataset and dataloader**
dataset = ECGMultiLeadDatasetWithLabels(ecg_lead_tensors, labels)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# **Encoder Model**
class ECGLeadEncoder(nn.Module):
    def __init__(self, input_dim=5000, latent_dim=256):
        super(ECGLeadEncoder, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()

        # Calculate output size after convolutions
        conv_output_dim = input_dim // (2 ** 3)  # 3 layers with stride=2
        self.fc_latent = nn.Linear(64 * conv_output_dim, latent_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.flatten(x)
        latent = self.fc_latent(x)
        return latent

# **Classifier Model Class**
class ECGLeadClassifier(nn.Module):
    def __init__(self, num_classes, use_12_leads=False, input_dim=5000, latent_dim=256):
        super(ECGLeadClassifier, self).__init__()

        # Define lead names
        self.use_12_leads = use_12_leads
        self.lead_names = (
            ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVF", "LEAD_aVL",
             "LEAD_V1", "LEAD_V2", "LEAD_V3", "LEAD_V4", "LEAD_V5", "LEAD_V6"]
            if use_12_leads
            else ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVF", "LEAD_aVL"]
        )

        # Define encoders for each lead
        self.lead_encoders = nn.ModuleList([ECGLeadEncoder(input_dim=input_dim, latent_dim=latent_dim) for _ in self.lead_names])
        self.feature_dim = latent_dim

        # Define classifier network
        self.classifier = nn.Sequential(
            nn.Linear(len(self.lead_names) * self.feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, lead_data):
        lead_features = []
        for lead_name, encoder in zip(self.lead_names, self.lead_encoders):
            latent = encoder(lead_data[lead_name])
            lead_features.append(latent)

        combined_features = torch.cat(lead_features, dim=1)
        logits = self.classifier(combined_features)
        return logits

# **Training Function**
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            lead_data, labels = batch
            labels = labels.to(device)

            lead_data = {lead: lead_data[lead].to(device) for lead in lead_data}
            optimizer.zero_grad()
            outputs = model(lead_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

# **Evaluation Function**
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            lead_data, labels = batch
            labels = labels.to(device)

            lead_data = {lead: lead_data[lead].to(device) for lead in lead_data}
            logits = model(lead_data)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    auc_score = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return accuracy, auc_score, f1, mcc

# **Cross-Validation Training**
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

    # Initialize classifier model
    model = ECGLeadClassifier(num_classes=2, use_12_leads=False, input_dim=5000, latent_dim=256).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    # Train and evaluate
    train_classifier(model, train_loader, criterion, optimizer, epochs=50)
    accuracy, auc_score, f1, mcc = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score, f1, mcc))

    print(f'Fold {fold} Results: Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, F1: {f1:.4f}, MCC: {mcc:.4f}\n')

# **Calculate Metrics Across Folds**
accuracies, aucs, f1s, mccs = zip(*fold_results)
print(f'Mean Accuracy: {np.mean(accuracies):.4f}, STD: {np.std(accuracies):.4f}')
print(f'Mean AUC: {np.mean(aucs):.4f}, STD: {np.std(aucs):.4f}')
print(f'Mean F1: {np.mean(f1s):.4f}, STD: {np.std(f1s):.4f}')
print(f'Mean MCC: {np.mean(mccs):.4f}, STD: {np.std(mccs):.4f}')


Task-2 (PAWP)

In [ ]:
import torch
import random
import numpy as np
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
import torch.nn as nn
import torch.optim as optim

# **Set device configuration**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# **Set seed for reproducibility**
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(123)

# Define file paths for the 12-lead ECG data and labels
lead_file_paths = {
    "LEAD_I": "../../Data_processing/data_aspire_PAWP_1/LEAD_I.pt",
    "LEAD_II": "../../Data_processing/data_aspire_PAWP_1/LEAD_II.pt",
    "LEAD_III": "../../Data_processing/data_aspire_PAWP_1/LEAD_III.pt",
    "LEAD_aVR": "../../Data_processing/data_aspire_PAWP_1/LEAD_aVR.pt",
    "LEAD_aVL": "../../Data_processing/data_aspire_PAWP_1/LEAD_aVL.pt",
    "LEAD_aVF": "../../Data_processing/data_aspire_PAWP_1/LEAD_aVF.pt",
    "LEAD_V1": "../../Data_processing/data_aspire_PAWP_1/LEAD_V1.pt",
    "LEAD_V2": "../../Data_processing/data_aspire_PAWP_1/LEAD_V2.pt",
    "LEAD_V3": "../../Data_processing/data_aspire_PAWP_1/LEAD_V3.pt",
    "LEAD_V4": "../../Data_processing/data_aspire_PAWP_1/LEAD_V4.pt",
    "LEAD_V5": "../../Data_processing/data_aspire_PAWP_1/LEAD_V5.pt",
    "LEAD_V6": "../../Data_processing/data_aspire_PAWP_1/LEAD_V6.pt"
}
labels_file_path = "../../Data_processing/data_aspire_PAWP_1/labels.pt"

# Load all lead tensors and labels
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Ensure all leads have the same number of samples
sample_count = len(next(iter(ecg_lead_tensors.values())))
assert len(labels) == sample_count, "Mismatch between number of labels and samples."
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count, "All leads must have the same number of samples."

# **Dataset Class**
class ECGMultiLeadDatasetWithLabels(Dataset):
    def __init__(self, ecg_leads, labels):
        self.ecg_leads = ecg_leads
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        lead_data = {lead: self.ecg_leads[lead][idx].unsqueeze(0) for lead in self.ecg_leads}
        label = self.labels[idx]
        return lead_data, label

# **Initialize the dataset and dataloader**
dataset = ECGMultiLeadDatasetWithLabels(ecg_lead_tensors, labels)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# **Encoder Model**
class ECGLeadEncoder(nn.Module):
    def __init__(self, input_dim=5000, latent_dim=256):
        super(ECGLeadEncoder, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()

        # Calculate output size after convolutions
        conv_output_dim = input_dim // (2 ** 3)  # 3 layers with stride=2
        self.fc_latent = nn.Linear(64 * conv_output_dim, latent_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.flatten(x)
        latent = self.fc_latent(x)
        return latent

# **Classifier Model Class**
class ECGLeadClassifier(nn.Module):
    def __init__(self, num_classes, use_12_leads=False, input_dim=5000, latent_dim=256):
        super(ECGLeadClassifier, self).__init__()

        # Define lead names
        self.use_12_leads = use_12_leads
        self.lead_names = (
            ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVF", "LEAD_aVL",
             "LEAD_V1", "LEAD_V2", "LEAD_V3", "LEAD_V4", "LEAD_V5", "LEAD_V6"]
            if use_12_leads
            else ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVF", "LEAD_aVL"]
        )

        # Define encoders for each lead
        self.lead_encoders = nn.ModuleList([ECGLeadEncoder(input_dim=input_dim, latent_dim=latent_dim) for _ in self.lead_names])
        self.feature_dim = latent_dim

        # Define classifier network
        self.classifier = nn.Sequential(
            nn.Linear(len(self.lead_names) * self.feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, lead_data):
        lead_features = []
        for lead_name, encoder in zip(self.lead_names, self.lead_encoders):
            latent = encoder(lead_data[lead_name])
            lead_features.append(latent)

        combined_features = torch.cat(lead_features, dim=1)
        logits = self.classifier(combined_features)
        return logits

# **Training Function**
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            lead_data, labels = batch
            labels = labels.to(device)

            lead_data = {lead: lead_data[lead].to(device) for lead in lead_data}
            optimizer.zero_grad()
            outputs = model(lead_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

# **Evaluation Function**
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            lead_data, labels = batch
            labels = labels.to(device)

            lead_data = {lead: lead_data[lead].to(device) for lead in lead_data}
            logits = model(lead_data)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    auc_score = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return accuracy, auc_score, f1, mcc

# **Cross-Validation Training**
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

    # Initialize classifier model
    model = ECGLeadClassifier(num_classes=2, use_12_leads=False, input_dim=5000, latent_dim=256).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    # Train and evaluate
    train_classifier(model, train_loader, criterion, optimizer, epochs=50)
    accuracy, auc_score, f1, mcc = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score, f1, mcc))

    print(f'Fold {fold} Results: Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, F1: {f1:.4f}, MCC: {mcc:.4f}\n')

# **Calculate Metrics Across Folds**
accuracies, aucs, f1s, mccs = zip(*fold_results)
print(f'Mean Accuracy: {np.mean(accuracies):.4f}, STD: {np.std(accuracies):.4f}')
print(f'Mean AUC: {np.mean(aucs):.4f}, STD: {np.std(aucs):.4f}')
print(f'Mean F1: {np.mean(f1s):.4f}, STD: {np.std(f1s):.4f}')
print(f'Mean MCC: {np.mean(mccs):.4f}, STD: {np.std(mccs):.4f}')


Taks-3

In [ ]:
import torch
import random
import numpy as np
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
import torch.nn as nn
import torch.optim as optim

# **Set device configuration**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# **Set seed for reproducibility**
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


# File paths
lead_file_paths = {
    "LEAD_I": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_I.pt",
    "LEAD_II": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_II.pt",
    "LEAD_III": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_III.pt",
    "LEAD_aVR": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVR.pt",
    "LEAD_aVL": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVL.pt",
    "LEAD_aVF": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVF.pt",
    "LEAD_V1": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V1.pt",
    "LEAD_V2": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V2.pt",
    "LEAD_V3": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V3.pt",
    "LEAD_V4": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V4.pt",
    "LEAD_V5": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V5.pt",
    "LEAD_V6": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V6.pt"
}
labels_file_path = "D:/ukbiobank/ECG_PAWP_UKB_Final/labels.pt"

# Load all lead tensors and labels
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Ensure all leads have the same number of samples as the labels
sample_count = len(next(iter(ecg_lead_tensors.values())))
print(len(labels))
print(sample_count)
assert len(labels) == sample_count, "Mismatch between number of labels and samples."
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count, "All leads must have the same number of samples."

# Define the dataset class
class ECGMultiLeadDatasetWithLabels(Dataset):
    def __init__(self, ecg_leads, labels):
        self.ecg_leads = ecg_leads
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        # Return each lead sample with the correct input shape and corresponding label
        lead_data = {lead: self.ecg_leads[lead][idx].unsqueeze(0) for lead in self.ecg_leads}
        label = self.labels[idx]
        return lead_data, label

# Initialize the dataset and dataloader
dataset = ECGMultiLeadDatasetWithLabels(ecg_lead_tensors, labels)

# **Encoder Model**
class ECGLeadEncoder(nn.Module):
    def __init__(self, input_dim=5000, latent_dim=256):
        super(ECGLeadEncoder, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()

        # Calculate output size after convolutions
        conv_output_dim = input_dim // (2 ** 3)  # 3 layers with stride=2
        self.fc_latent = nn.Linear(64 * conv_output_dim, latent_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.flatten(x)
        latent = self.fc_latent(x)
        return latent

# **Classifier Model Class**
class ECGLeadClassifier(nn.Module):
    def __init__(self, num_classes, use_12_leads=False, input_dim=5000, latent_dim=256):
        super(ECGLeadClassifier, self).__init__()

        # Define lead names
        self.use_12_leads = use_12_leads
        self.lead_names = (
            ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVF", "LEAD_aVL",
             "LEAD_V1", "LEAD_V2", "LEAD_V3", "LEAD_V4", "LEAD_V5", "LEAD_V6"]
            if use_12_leads
            else ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVF", "LEAD_aVL"]
        )

        # Define encoders for each lead
        self.lead_encoders = nn.ModuleList([ECGLeadEncoder(input_dim=input_dim, latent_dim=latent_dim) for _ in self.lead_names])
        self.feature_dim = latent_dim

        # Define classifier network
        self.classifier = nn.Sequential(
            nn.Linear(len(self.lead_names) * self.feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, lead_data):
        lead_features = []
        for lead_name, encoder in zip(self.lead_names, self.lead_encoders):
            latent = encoder(lead_data[lead_name])
            lead_features.append(latent)

        combined_features = torch.cat(lead_features, dim=1)
        logits = self.classifier(combined_features)
        return logits

# **Training Function**
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            lead_data, labels = batch
            labels = labels.to(device)

            lead_data = {lead: lead_data[lead].to(device) for lead in lead_data}
            optimizer.zero_grad()
            outputs = model(lead_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

# **Evaluation Function**
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            lead_data, labels = batch
            labels = labels.to(device)

            lead_data = {lead: lead_data[lead].to(device) for lead in lead_data}
            logits = model(lead_data)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    auc_score = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return accuracy, auc_score, f1, mcc

# **Cross-Validation Training**
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

    # Initialize classifier model
    model = ECGLeadClassifier(num_classes=2, use_12_leads=False, input_dim=5000, latent_dim=256).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    # Train and evaluate
    train_classifier(model, train_loader, criterion, optimizer, epochs=50)
    accuracy, auc_score, f1, mcc = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score, f1, mcc))

    print(f'Fold {fold} Results: Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, F1: {f1:.4f}, MCC: {mcc:.4f}\n')

# **Calculate Metrics Across Folds**
accuracies, aucs, f1s, mccs = zip(*fold_results)
print(f'Mean Accuracy: {np.mean(accuracies):.4f}, STD: {np.std(accuracies):.4f}')
print(f'Mean AUC: {np.mean(aucs):.4f}, STD: {np.std(aucs):.4f}')
print(f'Mean F1: {np.mean(f1s):.4f}, STD: {np.std(f1s):.4f}')
print(f'Mean MCC: {np.mean(mccs):.4f}, STD: {np.std(mccs):.4f}')
